In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
import scipy.stats
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

## Gather Data

coi data is collected from here: http://data.diversitydatakids.org/dataset/coi20-child-opportunity-index-2-0-database

locs data is collected from here: https://data.cdc.gov/Environmental-Health-Toxicology/Daily-Census-Tract-Level-PM2-5-Concentrations-2011/fcqm-xrf4

In [2]:
#data from coi
coi = pd.read_csv('coi.csv')

# to get the latitude,longitude for our coi data
locs = pd.read_csv('Daily_Census_Tract-Level_Ozone_Concentrations__2011-2014.csv')

#### How many unique census tracts are there in coi data for dallas,collin,tarrant and denton?

In [3]:
coi[coi['countyfips'].isin([48113,48085,48121,48439])].geoid.nunique() 

1172

Merging the coi data with locs to get the latitude and longitude

In [4]:
#merge these data files
coi_locs = pd.merge(coi,locs[((locs['date'] == '01JAN2011') & (locs['statefips'] == 48))][['ctfips','latitude','longitude']],how = 'inner',left_on = 'geoid', right_on = 'ctfips')

In [5]:
coi_locs.shape #for 2010 and 2015

(2344, 42)

Number of rows is 2344 because the data contains both 2010 and 2015

Collected the below data from
CDC Health Data at census tract level: https://data.cdc.gov/500-Cities/500-Cities-Census-Tract-level-Data-GIS-Friendly-Fo/k86t-wghb

In the 500-cities data we have physical health and mental health which are our y-variables.

In [6]:
df_500 = pd.read_excel('500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2019_release.xlsx')
print(df_500.shape)

(949, 64)


In [7]:
#filter coi only for 2015
df_coi_2015 = coi_locs[coi_locs['year'] == 2015]
df_coi_2015.shape

(1172, 42)

#### Selceting the required variables based on the correlations and studies related to child poverty.

In [8]:
#selecting the required columns
df_coi_2015 = df_coi_2015[['HE_FOOD', 'HE_GREEN', 'HE_WALK', 'HE_VACANCY', 'HE_SUPRFND', 'HE_RSEI',
       'HE_PM25', 'HE_OZONE', 'HE_HEAT', 'HE_HLTHINS','latitude', 'longitude','geoid','countyfips']]

df_500 = df_500[['TractFIPS', 'County', 'StateAbbr', 'PlaceName',
       'PlaceFIPS', 'Place_TractID', 'Population2010', 'ACCESS2_CrudePrev',
       'ARTHRITIS_CrudePrev', 'BINGE_CrudePrev', 'BPHIGH_CrudePrev',
       'BPMED_CrudePrev', 'CANCER_CrudePrev', 'CASTHMA_CrudePrev',
       'CHD_CrudePrev', 'CHECKUP_CrudePrev', 'CHOLSCREEN_CrudePrev',
       'COLON_SCREEN_CrudePrev', 'COPD_CrudePrev', 'COREM_CrudePrev',
       'COREW_CrudePrev', 'CSMOKING_CrudePrev', 'DENTAL_CrudePrev',
       'DIABETES_CrudePrev', 'HIGHCHOL_CrudePrev', 'KIDNEY_CrudePrev',
       'LPA_CrudePrev', 'MAMMOUSE_CrudePrev', 'MHLTH_CrudePrev',
       'OBESITY_CrudePrev', 'PAPTEST_CrudePrev', 'PHLTH_CrudePrev',
       'SLEEP_CrudePrev', 'STROKE_CrudePrev', 'TEETHLOST_CrudePrev']]

print(df_coi_2015.shape)
print(df_500.shape)

(1172, 14)
(949, 35)


How many unique census tracts in 500-cities data file that are in coi data?

In [9]:
df_coi_2015['geoid'].isin(df_500.TractFIPS.values).sum()

920

Total rows in the 500-cities are 949 but unique census tracts are 920. So there are duplicate census tracts in the data.

What are the Duplicate rows?

In [10]:
df_500[df_500.duplicated(subset = ['TractFIPS'], keep = False)].sort_values(by = 'TractFIPS').head(2)

,TractFIPS,County,StateAbbr,PlaceName,PlaceFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,...,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev
627,48085030522,Collin,TX,Frisco,4827684,4827684-48085030522,1363,15.0,13.6,19.4,...,1.8,22.6,79.3,9.7,27.3,82.0,7.5,34.9,1.5,6.2
817,48085030522,Collin,TX,McKinney,4845744,4845744-48085030522,2820,15.8,11.3,23.5,...,1.5,21.4,78.6,10.5,28.0,83.8,6.9,33.7,1.1,5.8


There are 29 duplicates in the 500-cities data, above is just an example of one duplicate. From here, we can observe even though the census tract is same but the variables data are different.

To resolve this we have imputed by averaging the values from both rows and dropped the duplicate.

In [11]:
#remove the duplicates by imputing there average.
impute_cols = ['ACCESS2_CrudePrev',
       'ARTHRITIS_CrudePrev', 'BINGE_CrudePrev', 'BPHIGH_CrudePrev',
       'BPMED_CrudePrev', 'CANCER_CrudePrev', 'CASTHMA_CrudePrev',
       'CHD_CrudePrev', 'CHECKUP_CrudePrev', 'CHOLSCREEN_CrudePrev',
       'COLON_SCREEN_CrudePrev', 'COPD_CrudePrev', 'COREM_CrudePrev',
       'COREW_CrudePrev', 'CSMOKING_CrudePrev', 'DENTAL_CrudePrev',
       'DIABETES_CrudePrev', 'HIGHCHOL_CrudePrev', 'KIDNEY_CrudePrev',
       'LPA_CrudePrev', 'MAMMOUSE_CrudePrev', 'MHLTH_CrudePrev',
       'OBESITY_CrudePrev', 'PAPTEST_CrudePrev', 'PHLTH_CrudePrev',
       'SLEEP_CrudePrev', 'STROKE_CrudePrev', 'TEETHLOST_CrudePrev']

df_500[impute_cols] = df_500[impute_cols + ['TractFIPS']].groupby('TractFIPS').transform('mean')

In [12]:
df_500.drop_duplicates(subset = ['TractFIPS'], inplace = True)

In [13]:
#check duplicates
df_500[df_500.duplicated(subset = ['TractFIPS'], keep = False)].sort_values(by = 'TractFIPS')

,TractFIPS,County,StateAbbr,PlaceName,PlaceFIPS,Place_TractID,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,...,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev


Merging the coi data and 500-cities data

In [14]:
#merge data frames this is our master dataframe from this we can select whatever the variables we want
df_merge = pd.merge(df_coi_2015,df_500,how = 'left', left_on = 'geoid', right_on = 'TractFIPS')
df_merge.shape

(1172, 49)

In [15]:
# renaming the columns to represent the coi variables
df_merge.rename({'HE_FOOD': 'COI_FOOD',
       'HE_GREEN': 'COI_GREEN','HE_WALK' : 'COI_WALK','HE_VACANCY' : 'COI_VACANCY','HE_SUPRFND' :'COI_SUPRFND',
                 'HE_RSEI' : 'COI_RSEI',
       'HE_PM25' : 'COI_PM25','HE_OZONE' : 'COI_OZONE','HE_HEAT' : 'COI_HEAT','HE_HLTHINS' : 'COI_HLTHINS'},
                axis = 1, inplace = True)

In [16]:
df_merge.columns

Index(['COI_FOOD', 'COI_GREEN', 'COI_WALK', 'COI_VACANCY', 'COI_SUPRFND',
       'COI_RSEI', 'COI_PM25', 'COI_OZONE', 'COI_HEAT', 'COI_HLTHINS',
       'latitude', 'longitude', 'geoid', 'countyfips', 'TractFIPS', 'County',
       'StateAbbr', 'PlaceName', 'PlaceFIPS', 'Place_TractID',
       'Population2010', 'ACCESS2_CrudePrev', 'ARTHRITIS_CrudePrev',
       'BINGE_CrudePrev', 'BPHIGH_CrudePrev', 'BPMED_CrudePrev',
       'CANCER_CrudePrev', 'CASTHMA_CrudePrev', 'CHD_CrudePrev',
       'CHECKUP_CrudePrev', 'CHOLSCREEN_CrudePrev', 'COLON_SCREEN_CrudePrev',
       'COPD_CrudePrev', 'COREM_CrudePrev', 'COREW_CrudePrev',
       'CSMOKING_CrudePrev', 'DENTAL_CrudePrev', 'DIABETES_CrudePrev',
       'HIGHCHOL_CrudePrev', 'KIDNEY_CrudePrev', 'LPA_CrudePrev',
       'MAMMOUSE_CrudePrev', 'MHLTH_CrudePrev', 'OBESITY_CrudePrev',
       'PAPTEST_CrudePrev', 'PHLTH_CrudePrev', 'SLEEP_CrudePrev',
       'STROKE_CrudePrev', 'TEETHLOST_CrudePrev'],
      dtype='object')

In [17]:
df_merge.shape

(1172, 49)

loading the life expectancy data(Y-variable) and merging with our master data

In [18]:
#load cdc life expectancy
df_life_expectancy = pd.read_csv('CDC Life Expectancy.csv')
print(df_life_expectancy.shape)

df_merge = pd.merge(df_merge,df_life_expectancy,how = 'left',left_on = 'geoid', right_on = 'Tract ID')
df_merge.drop('Tract ID', axis = 1,inplace = True)
print(df_merge.shape)

(999, 2)
(1172, 50)


Loading traffic data the data is collected from :https://cris.dot.state.tx.us/public/Query/app/public/query/

In [19]:
#Load input data(traffic) and merge with our master data
df_traffic = pd.read_excel('input.xlsx')
df_traffic.shape

(1172, 23)

In [20]:
df_merge = pd.merge(df_merge,df_traffic[['geoid','Alcohol Test', 'Drug Test', 'Pedalcyclist',
       'Pedastrian']],how = 'left',on = 'geoid')
#df_merge.drop('Tract ID', axis = 1,inplace = True)
print(df_merge.shape)
print(df_merge.columns)

(1172, 54)
Index(['COI_FOOD', 'COI_GREEN', 'COI_WALK', 'COI_VACANCY', 'COI_SUPRFND',
       'COI_RSEI', 'COI_PM25', 'COI_OZONE', 'COI_HEAT', 'COI_HLTHINS',
       'latitude', 'longitude', 'geoid', 'countyfips', 'TractFIPS', 'County',
       'StateAbbr', 'PlaceName', 'PlaceFIPS', 'Place_TractID',
       'Population2010', 'ACCESS2_CrudePrev', 'ARTHRITIS_CrudePrev',
       'BINGE_CrudePrev', 'BPHIGH_CrudePrev', 'BPMED_CrudePrev',
       'CANCER_CrudePrev', 'CASTHMA_CrudePrev', 'CHD_CrudePrev',
       'CHECKUP_CrudePrev', 'CHOLSCREEN_CrudePrev', 'COLON_SCREEN_CrudePrev',
       'COPD_CrudePrev', 'COREM_CrudePrev', 'COREW_CrudePrev',
       'CSMOKING_CrudePrev', 'DENTAL_CrudePrev', 'DIABETES_CrudePrev',
       'HIGHCHOL_CrudePrev', 'KIDNEY_CrudePrev', 'LPA_CrudePrev',
       'MAMMOUSE_CrudePrev', 'MHLTH_CrudePrev', 'OBESITY_CrudePrev',
       'PAPTEST_CrudePrev', 'PHLTH_CrudePrev', 'SLEEP_CrudePrev',
       'STROKE_CrudePrev', 'TEETHLOST_CrudePrev', 'life expectancy',
       'Alcohol Test',

In [30]:
df_merge.to_csv('master_raw_data.csv')